In [11]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier as OVR

In [12]:
def transform_y(y_t, class_num):
    # print(class_num)
    y = np.array(y_t)
    y[y == class_num] = -1
    y[y != -1] = 1
    return y

In [13]:
def calc_slack(plane, svid):
    slack = np.zeros_like(plane)
    for i in svid:
        # round to 2-digit right to the decimal point
        slack[i] = np.around(abs(1-plane[i]), 3)  
    return slack

In [14]:
# -------------data preparation-------------
df_train = pd.read_table("train.txt")
df_test = pd.read_table("test.txt")
# print(df)
data_train = df_train.to_numpy()
data_test = df_test.to_numpy()

x_train = np.delete(data_train, [0], axis=1)
y_train = data_train[:, 0]

x_test = np.delete(data_test, [0], axis=1)
y_test = data_test[:, 0]

y_train_Setosa = transform_y(y_train, 0)
y_train_Versicolour = transform_y(y_train, 1)
y_train_Virginica = transform_y(y_train, 2)

y_class = [y_train_Setosa, y_train_Versicolour, y_train_Virginica]

In [15]:
# -------------SVM Modelling-------------
with open("SVM_slack.txt", 'w') as f:
    for i in range(1, 11):
        my_C = i/10
        ovr_clf = OVR(SVC(C=my_C, kernel="linear")).fit(x_train, y_train)
        training_error = 1 - ovr_clf.score(x_train, y_train)
        testing_error = 1 - ovr_clf.score(x_test, y_test)
#         print("\nC =", my_C)
#         print(training_error)
#         print(testing_error)
        
        f.write("C:")
        f.write(str(my_C))
        f.write("\n")
        f.write(str(training_error))
        f.write("\n")
        f.write(str(testing_error))
        f.write("\n")

        for j in y_class:
            clf = SVC(C=my_C, kernel="linear").fit(x_train, j)
            w = clf.coef_[0]
            b = clf.intercept_[0]
            svc_num = clf.n_support_[0]
            svid = np.sort(np.array(clf.support_[:]), axis = None)
            __plane = clf.decision_function(x_train) * j
            slack = calc_slack(__plane, svid)

            w_ = ', '.join([str(k) for k in w])
            f.write(w_)
            f.write("\n")
            f.write(str(b))
            f.write("\n")
            id1 = ', '.join([str(k) for k in svid])
            f.write(id1)
            f.write("\n")
            slk = ', '.join([str(k) for k in slack])
            f.write(slk)
            f.write("\n")
        f.write("\n")
        f.write("\n")

        
f.close()